In [19]:
import sqlite3
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

# Configuration
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [2]:
class DatabaseManager:
    """Gerencia conexões e operações com bancos de dados."""

    @staticmethod
    def get_database_connection():
        """Gerenciador de contexto para conexões de banco de dados com limpeza adequada de recursos."""

        # Pega a pasta raiz do projeto
        notebook_dir = Path.cwd()
        project_root = (
            notebook_dir.parent
            if notebook_dir.name == "notebooks"
            else notebook_dir
        )

        # Conexão com SQLite
        db_path = project_root / "data" / "db" / "database.db"

        # Garantir que o diretório exista
        db_dir = db_path.parent
        if not db_dir.exists():
            db_dir.mkdir(parents=True)

        conn = None
        try:
            conn = sqlite3.connect(db_path)
            yield conn
        except Exception as e:
            logger.error(f"Erro ao conectar com o banco de dados: {e}")
            raise
        finally:
            if conn:
                conn.close()

    def execute_query(self, query, params=None):
        """Executa uma consulta no SQLite."""
        try:
            cursor = self.conn.cursor()
            cursor.execute(query, params or ())
            result = cursor.fetchall()
            return result
        except Exception as e:
            self.logger.error(f"Falha ao executar consulta: {e}")
            return None

    def disconnect(self):
        """Fecha todas as conexões."""
        if self.conn:
            self.conn.close()


In [3]:
# Configurar pandas para exibir números sem notação científica
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.precision", 2)

# Carregar dados do Banco
db_manager = DatabaseManager()
for conn in db_manager.get_database_connection():
    query = "SELECT * FROM vw_aai"
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]
    df = pd.DataFrame.from_records(cursor.fetchall(), columns=columns)

if df["data_referencia"].dtype == "object":
    df["data_referencia"] = pd.to_datetime(
        df["data_referencia"], format="%Y-%m-%d"
    )

le = LabelEncoder()
for column in df.select_dtypes(include=["object"]).columns:
    df[column] = le.fit_transform(df[column])

# Criar cópia mascarada para visualização (valores sensíveis ocultos)
df_display = df.copy()


def mask_value(x):
    """Mascara valores numéricos mantendo apenas os 2 primeiros dígitos"""
    if pd.isna(x) or x == 0:
        return x
    str_val = f"{abs(x):.0f}"
    if len(str_val) > 2:
        masked = str_val[:2] + "*" * (len(str_val) - 2)
        return masked if x >= 0 else f"-{masked}"
    return x


# Aplicar máscara apenas em colunas numéricas (exceto data_referencia e codigo_assessor)
for column in df_display.select_dtypes(include=["float64", "int64"]).columns:
    if column not in ["data_referencia", "codigo_assessor"]:
        df_display[column] = df[column].apply(mask_value)

print(df_display.head())
print(f"\nShape: {df.shape}")


  data_referencia  codigo_assessor  net_total net_renda_fixa  \
0      2025-10-01                0   72******       42******   
1      2025-10-01                1   93******       51******   
2      2025-10-01                2  26*******      10*******   
3      2025-10-01                3   11******        31*****   
4      2025-10-01                4    75*****        36*****   

  net_fundos_imobiliarios net_renda_variavel net_fundos net_financeiro  \
0                 32*****            76*****   16******         27****   
1                 25*****           15******   19******         61****   
2                 62*****           49******   92******        14*****   
3                 37*****            11*****    35*****          27***   
4                 23*****             37****     75****          44***   

  net_previdencia net_outros  ... receita_aluguel_total  \
0          73****    18*****  ...                   27*   
1         30*****     94****  ...                   

In [33]:
# Criar dashboard com plotly
# Configurar estilo dos gráficos
sns.set_style("whitegrid")
plt.rcParams["figure.facecolor"] = "white"

# Criar coluna mascarada para visualização
df_plot = df.copy()
df_plot = df_plot.sort_values(by="net_total", ascending=False).head(20)
df_plot["net_total_masked"] = df_plot["net_total"].apply(mask_value)
df_plot["codigo_assessor"] = df_plot["codigo_assessor"].astype(str)

# Visualizamos net total por assessor (ordenado do maior para o menor)
fig = px.bar(
    df_plot,
    x="codigo_assessor",
    y="net_total",  # Usar valor real para plotagem
    title="Net Total por Código de Assessor (Top 20)",
    labels={"codigo_assessor": "Código do Assessor"},
    text="net_total_masked",  # Mostrar valores mascarados nos rótulos
    custom_data=["net_total_masked"],  # Dados customizados para hover
)

# Atualizar layout para melhor visualização
fig.update_traces(
    textposition="outside",
    hovertemplate="<b>Código do Assessor:</b> %{x}<br>"
    + "<b>Net Total:</b> %{customdata[0]}<br>"
    + "<extra></extra>",
)
fig.update_layout(
    xaxis={"categoryorder": "total descending"},  # Forçar ordem descendente
    yaxis_title="Net Total",
    yaxis=dict(
        showticklabels=False,  # Esconder valores no eixo Y
        title="Net Total",
    ),
    height=600,
    showlegend=False,
)

fig.show()


In [34]:
# Criar coluna mascarada para visualização
df_plot = df.copy()
df_plot = df_plot.sort_values(by="saldo_cliente_medio", ascending=False).head(20)
df_plot["saldo_cliente_medio"] = df_plot["saldo_cliente_medio"].apply(mask_value)
df_plot["codigo_assessor"] = df_plot["codigo_assessor"].astype(str)

# Visualizamos saldo médio por assessor (ordenado do maior para o menor)
fig = px.bar(
    df_plot,
    x="codigo_assessor",
    y="saldo_cliente_medio",  # Usar valor real para plotagem
    title="Saldo Médio por Código de Assessor (Top 20)",
    labels={"codigo_assessor": "Código do Assessor"},
    text="saldo_cliente_medio",  # Mostrar valores mascarados nos rótulos
    custom_data=["saldo_cliente_medio"],  # Dados customizados para hover
)

# Atualizar layout para melhor visualização
fig.update_traces(
    textposition="outside",
    hovertemplate="<b>Código do Assessor:</b> %{x}<br>"
    + "<b>Saldo Médio:</b> %{customdata[0]}<br>"
    + "<extra></extra>",
)
fig.update_layout(
    xaxis={"categoryorder": "total descending"},  # Forçar ordem descendente
    yaxis_title="Saldo Médio",
    yaxis=dict(
        showticklabels=False,  # Esconder valores no eixo Y
        title="Saldo Médio",
    ),
    height=600,
    showlegend=False,
)

fig.show()
